Draw maps and mark all the McDonalnd's and Starbucks points

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
m = pd.read_excel('m_shops.xlsx', index_col=0)
s = pd.read_excel('s_shops.xlsx', index_col=0)

draw map

In [3]:
# draw the map with marks of 2 types

address = 'Hong Kong, China'

geolocator = Nominatim(user_agent="my_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add McDonald's markers to the map
markers_colors = []
for lat, lon in zip(m['lat'], m['lng']):
    label = folium.Popup("McDonald's", parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7).add_to(map_clusters)

# add Starbucks markers to the map
markers_colors = []
for lat, lon in zip(s['lat'], s['lng']):
    label = folium.Popup('Starbuck', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7).add_to(map_clusters)
    
    
map_clusters

In [4]:
m = m.reset_index(drop = True)
s = s.reset_index(drop = True)

There are some McDonald's shops and Starbucks' shops that are nearby. We will combine the 2 points if a McDonald's shop and Starbucks' shop are within 50 meter. The below function will create a list of 3 types location data: type 0 - McDonald's shop location, type 1 - Starbucks' shop location & type 2 - the location has both 2 shops

In [5]:
from math import radians, sin, cos, acos
import time
start = time.time()

default_distance = 50
s_shape, tmp = s.shape
m_shape, tmp = m.shape
s_mark = [True] * s_shape
m_mark = [True] * m_shape

cmp_list = [] 
# a list to store types (*), latitude and longitude
# type 0: McDonald's
# type 1: Starbucks
# type 2: Both

def dist(slat, slon, elat, elon):
    return(6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon)))

z = 0
for i, m_row in m.iterrows():
    for j, s_row in s.iterrows():
        if m_mark[i] and s_mark[j]:
            row_1 = [None] * 3
            row_2 = [None] * 3
            if dist(m_row['lat'], m_row['lng'], s_row['lat'], s_row['lng']) < default_distance:
                row_1 = [2, (m_row['lat']+s_row['lat'])/2 , (m_row['lng']+s_row['lng'])/2]
                cmp_list.append(row_1)
                m_mark[i] = False
                s_mark[j] = False
            
for i, m_row in m.iterrows():
    if m_mark[i]:
        row_1 = [0, m_row['lat'], m_row['lng']]
        cmp_list.append(row_1)
            
for j, s_row in s.iterrows():
    if s_mark[j]:
        row_2 = [1, s_row['lat'], s_row['lng']]
        cmp_list.append(row_2)
        
end = time.time()
end - start

8.338258028030396

create a full list of all shops locations

In [6]:
whole_list = pd.DataFrame(data=cmp_list, columns=['Type','Latitude','Longitude'])

In [7]:
# draw the map with the marks of 3 types

address = 'Hong Kong, China'

geolocator = Nominatim(user_agent="my_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

clr = ['red','blue','green']
string = ["McDonald's", "Starbucks", "Both"]

# add markers to the map
markers_colors = []
for lat, lon, t in zip(whole_list['Latitude'], whole_list['Longitude'], whole_list['Type']):
    label = folium.Popup(string[t], parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=clr[t],
        fill=True,
        fill_color=clr[t],
        fill_opacity=0.7).add_to(map_clusters)

        
map_clusters

In [8]:
whole_list.to_excel('whole.xlsx')

In [9]:
whole_list.head()

,Type,Latitude,Longitude
0,2,22.285922,114.192492
1,2,22.278530,114.172924
2,2,22.318625,114.168536
3,2,22.289643,114.197157
4,2,22.304270,114.189631
